In [1]:
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
import random
import statistics as stats
import warnings
import time
import math
import pandas as pd
from google.colab import files

In [2]:
def plot(digraph):
    
    plt.rcParams["figure.figsize"] = [12, 12]
    plt.rcParams["figure.autolayout"] = True
    #pos = nx.spectral_layout(digraph)
    pos = nx.circular_layout(digraph)
   
    plt.show()
    warnings.filterwarnings("ignore")
    nx.draw(digraph, pos,with_labels=True,node_color='blue',font_color='white')

    

In [3]:
#different schedule functions for temperature
def scheduleAlpha(t, temperature):
    return temperature/(t+1)


def schedulePower(t, temperature):
    return (1/math.pow(2,t))*temperature
    
    
def scheduleLevel(t, temperature, d = 2):
    if (t%d == 0):
        return temperature - 0.5*temperature
    return temperature

In [4]:
#returns a randomly generated neighbor, by removing or adding a node to the given state
def generatePossibleNeighbor(state, initialStateCopy):
    stateNodes = list(state.nodes())
    stateCopy = state.copy()
    
    addOrRemoveNode = np.random.uniform(0,1)
   
    if (addOrRemoveNode < 0.75 or initialStateCopy.number_of_nodes() == state.number_of_nodes()):
        nodeToRemove = random.choice(stateNodes)
        stateCopy.remove_node(nodeToRemove)
        print("\nnode removed : ", nodeToRemove)
   
    else:
        initialNodes = list(initialStateCopy.nodes())
        initialEdges = list(initialStateCopy.edges())
        nodeToAdd = random.choice([elem for elem in initialNodes if elem not in stateNodes])
        edgesToAdd = [elem for elem in initialEdges if (elem[0] == nodeToAdd and elem[1] in stateNodes) or (elem[1] == nodeToAdd and elem[0] in stateNodes)]
        stateCopy.add_edges_from(edgesToAdd)
        print("\nnode added : ", nodeToAdd)
   
    return stateCopy

In [5]:
# returns true if a path exists between u et v
def existPath(matriceAdj, u, v):
    n = len(matriceAdj) # number of nodes
    file = []
    visites = [False] * n
    file.append(u)
    while file:
        courant = file.pop(0)
        visites[courant] = True
        for i in range(n):
            if matriceAdj[courant,i] > 0 and visites[i] == False:
                file.append(i)
                visites[i] = True
            elif matriceAdj[courant,i] > 0 and i == v:
                return True 
    return False

In [6]:
# returns the number of nodes in at least one cycle in the given state
def nbNodesInAtLeastOneCycle(state):
    count = 0
    adjacencyMatrix=nx.adjacency_matrix(state)
    adjacencyMatrix=adjacencyMatrix.todense()
    for node in range(0,len(adjacencyMatrix)):
      if existPath(adjacencyMatrix, node, node):
        count+=1
    return count

In [7]:
def bestStateBetween(currentState, neighborState,temperature):
    # end because temperature = 0
    if (temperature == 0):
      return currentState
    # normal case
    delta_E = nbNodesInAtLeastOneCycle(neighborState) - nbNodesInAtLeastOneCycle(currentState)
    print("deltaE :", delta_E)
    if (delta_E < 0):
        return neighborState
    elif (delta_E == 0):
        if (neighborState.number_of_nodes() > currentState.number_of_nodes()):
            return neighborState
        else:
            return currentState
    else:
        proba = math.exp(-delta_E/temperature)
        trial = np.random.uniform(0,1)
        print ("temperature :", temperature, "/ -deltaE :", -delta_E, "/ proba :", proba, "/trial :", trial)
        if (trial < proba):
            return neighborState
        else:
            return currentState

In [8]:
def simulatedAnnealing(nbNodes, probasErdosRenyi, plotResults, scheduleNb = 1):
    startTime = time.time()

    nbNodes = nbNodes
    probaErdosRenyi = probasErdosRenyi
    initialState = nx.generators.random_graphs.erdos_renyi_graph(nbNodes,probaErdosRenyi, directed= True)
    initialStateCopy = initialState.copy()



    print("\n\nWe start with this initial graph : " , list(initialStateCopy.edges()))


    # simulated annealing algorithm
    temperature = initialStateCopy.number_of_nodes()*10000
    currentState = initialState
    t = 1
    while(temperature > 0 and nbNodesInAtLeastOneCycle(currentState) > 0):
        print("temperature: ",temperature)
        if (scheduleNb == 1):
          temperature = scheduleAlpha(t,temperature) 
        elif (scheduleNb == 2):
          temperature = scheduleLevel(t,temperature,2)
        elif (scheduleNb == 3):
          temperature = schedulePower(t,temperature) 
        neighborState = generatePossibleNeighbor(currentState, initialStateCopy)
        currentState = bestStateBetween(currentState, neighborState, temperature)
        t = t+1
        
        print("\ncurrent state : ", list(currentState.edges()))

    print("\n\n-------------------END OF ALGORITHM-------------------\n")
     
    try:
      print("Cycle in solution ? ",nx.find_cycle(currentState))
      remainingCycles = True
    except nx.exception.NetworkXNoCycle as e:
      print("Found no cycle ")
      remainingCycles = False



    # we made an acyclic graph out of the initial graph
    # now we must deduce the cycle cutter and display it
    cycleCutter = [elem for elem in list(initialStateCopy.nodes()) if elem not in list(currentState.nodes())]

    finishTime = time.time()
    executionTime = finishTime - startTime

    print("\n\n-------------------CONCLUSION-------------------\n\nThe initial graph had", nbNodes, "nodes and", nbNodesInAtLeastOneCycle(initialStateCopy), "nodes in at least one cycle.")
    print("\nAnd we obtained the following acyclic graph :\n\n" , list(currentState.edges()))
    print("\nThus the cycle-cutter is :", cycleCutter, ", the simulated annealing algorithm removed", len(cycleCutter), "nodes, with execution time =", round(executionTime,2), "seconds.")

    if (plotResults):
        print("\n\n-------------------INITIAL STATE (Fig 1) VS FINAL STATE (Fig 2)-------------------")
        plot(initialStateCopy)
        plot(currentState)

    return (len(cycleCutter), executionTime, remainingCycles)

In [ ]:
#Testez l’algorithme de Simulated Annealing dans cette cellule *********************************************************************
simulatedAnnealing(70, 0.04, True,1)

In [ ]:
# statistics
'''
nbOfNodesToConsider = [10,30,70,100]
probasErdosRenyiToConsider = [0.02,0.04,0.06,0.08,0.1,0.2,0.3,0.4,0.5]
scheduleToConsider = [1,2,3] # 1 for scheduleAlpha, 2 for scheduleLevel, 3 for schedulePower 
sampleSize = 15

for nbNode in nbOfNodesToConsider: 
  
  nodeListDf = []
  probaListDf = []
  scheduleListDf = []
  meanCycleCutterListDf = []
  standardDeviationCycleCutterListDf = []
  meanExecutionTimeListDf = []
  standarDeviationExecutionTimeListDf = []
  remainingCyclesCasesListDf = []

  for scheduleNb in scheduleToConsider:
    for proba in probasErdosRenyiToConsider:

        nbNodes = nbNode
        probaErdosRenyi = proba
        schedule = scheduleNb
        # plotResults = True only if trial for one graph, not in a loop for several graphs

        resultList = [0] * sampleSize
        executionTimeList = [0] * sampleSize
        #by construction, can finish with remaining cycles if temperature has reached 0 too soon
        nbCasesWhereRemainingCycles = 0
        for i in range(sampleSize):
          result = simulatedAnnealing(nbNodes,probaErdosRenyi,False,schedule)
          resultList[i] = result[0]
          executionTimeList[i] = result[1]
          if (result[2]):
            nbCasesWhereRemainingCycles+=1

        averageExecutionTime = round((sum(executionTimeList)/len(executionTimeList)),2)
        mean = round(sum(resultList)/len(resultList),2)
        standardDeviation = round(math.sqrt(stats.pvariance(resultList)),2)
        standardDeviationExecutionTime = round(math.sqrt(stats.pvariance(executionTimeList)),2)
        percentageCasesRemainingCycles = round((nbCasesWhereRemainingCycles/sampleSize),4) * 100

        print("\n\n-------------------MEAN & STANDARD DEVIATION-------------------\n")
        print("The initial graph had", nbNodes, "nodes and probability of Erdos-Renyi", probaErdosRenyi,".")
        print("\nIn average, the cycle-cutter contains :", mean, "nodes, with a standard deviation of :", standardDeviation,".")
        print("\nIn average, the execution time of the simulated annealing algorithm with the above parameters is :", averageExecutionTime, "seconds, with a standard deviation of :",standardDeviationExecutionTime,".")

         #completing lists for the dataframe
        nodeListDf.append(nbNode)
        probaListDf.append(proba)
        if (scheduleNb == 1):
          scheduleListDf.append("alpha")
        elif (scheduleNb == 2):
          scheduleListDf.append("level")
        elif (scheduleNb == 3):
          scheduleListDf.append("power")
        meanCycleCutterListDf.append(mean)
        standardDeviationCycleCutterListDf.append(standardDeviation)
        meanExecutionTimeListDf.append(averageExecutionTime)
        standarDeviationExecutionTimeListDf.append(standardDeviationExecutionTime)
        remainingCyclesCasesListDf.append(percentageCasesRemainingCycles)

  #creating a report for each number of nodes in graph
  results = {"nombre de sommets" : nodeListDf, "proba Erdos-Renyi" : probaListDf, "temperature schedule (alpha, level ou power)" : scheduleListDf, "moyenne taille coupe cycle" : meanCycleCutterListDf, "écart-type taille coupe cycle" : standardDeviationCycleCutterListDf, "moyenne temps exécution" : meanExecutionTimeListDf, "écart-type temps exécution" : standarDeviationExecutionTimeListDf, "pourcentage de cas présentant toujours des cycles à la fin de l'algorithme" : remainingCyclesCasesListDf}
  results_df = pd.DataFrame(results)
  moncsv = results_df.to_csv("results_" + str(nbNode) +".csv", sep=";")
  files.download("results_" + str(nbNode) +".csv") 
  '''